# Técnicas avanzadas de diseño de software

## Delayed based Vs Rollback netcode

### Manuel Nieto - Ignacio Ramirez - Bruno Volpini

![](imagenes/patos.png)

# Primero lo primero
## ¿Qué es el netcode?

En realidad, nada.

Netcode es un termino que se usa para hablar de manera muy general de todo lo que tiene que ver con la gestion de redes en los juegos en linea. Es un termino que se usa solamente en los juegos y segun Wikipedia
no es un término aceptado por la comunidad cientifica.

# Bueno, ¿y?

Los juegos multijugador en linea tienen un problema fundamental: la latencia.
## Latencia
Es el tiempo total que tarda un dato en ir desde tu dispositivo hasta el servidor del juego y regresar. Incluye absolutamente todo el proceso de transmisión de datos.

# ¿Como lo solucionamos?
Vamos a ver una evolución histórica de como se trato de solucionar este problema.

# En un principio solo estaba la Xband
Desarrollada por Catapult Entertainment en 1994, XBAND era un módem que permitía jugar online en Mega Drive y SNES.
![](imagenes/xband-1.jpg)

# Problema
¿Cómo registrar los botones que pulsaba cada jugador, enviarlos entre los dos modems, sincronizarlos, mostrarlos en pantalla y que todo se haga en un tiempo razonable?
A esta secuencia se la conoce como network delay.

# Solución 
Tratar de conectar a jugadores que tengan el mismo código de area y algunos juegos de pelea trataron de enfocar su combate en la complejidad y no tanto en la velocidad.

# Por que hacia falta un buen netcode?

 - Basicamente por que el problema de jugar en linea es el siguiente:

    ![](imagenes/xbandgif.gif)

### Ahora sí
# Delay Based Netcode
Durante finales de los 90s y principios de los 2000s no existía el concepto de delay based netcode como tal, sino que se hablaba de simplemente jugar en linea.

# Idea
- Esperar a que lleguen las entradas de los dos jugadores
- Una vez sincronizadas, ejecuta ambas
![](imagenes/delay_based.gif)

## Problema
- Si la latencia entre los jugadores es alta, perdes responsiveness

![](imagenes/delay_based_problem.gif)

# Hay que buscarle la vuelta
Como a mediados de los 90 no existían buenas conexiones a internet en las casas, el delay based netcode no era una opción viable.

En 1996 la empresa id-Software saca QuakeWorld, una continuación de Quake pensada exclusivamente para el multijugador por internet.
![](imagenes/Quake1cover.jpg)

## John Carmack

![](imagenes/johncarmack.png)

#### <sub><sup>Nos quedamos sin pixeles</sub></sup>


## La idea de Carmack
"<em>The biggest difference is the addition of client side movement simulation.</em>

<em>I am now allowing the client to guess at the results of the users movement 
until the authoritative response from the server comes through.  This is a 
biiiig architectural change.  The client now needs to know about solidity 
of objects, friction, gravity, etc.  I am sad to see the elegent 
client-as-terminal setup go away, but I am practical above idealistic.</em>

<em>The server is still the final word, so the client is allways repredicting 
it's movement based off of the last known good message from the server.</em>"

Artículo de John Carmack - 1996

Tener una estructura básica de comunicación:
```C
typedef struct
{
	vec3_t	viewangles;

// intended velocities
	float	forwardmove;
	float	sidemove;
	float	upmove;
#ifdef QUAKE2
	byte	lightlevel;
#endif
} usercmd_t;
```

Cada comando enviado al servidor se guarda en el arreglo frames:
```C
frame_t		frames[UPDATE_BACKUP]; // UPDATE_BACKUP = 64
```
```C
typedef struct
{
	// generated on client side
	usercmd_t	cmd;		// cmd that generated the frame
	double		senttime;	// time cmd was sent off
	int	delta_sequence;		// sequence number to delta from, -1 = full update

	// received from server
	double		receivedtime;	// time message was received, or -1
	player_state_t	playerstate[MAX_CLIENTS];	// player_state_t is the information needed by a player entity
                    							// to do move prediction and to generate a drawable entity
	packet_entities_t	packet_entities;
	qboolean	invalid;		// true if the packet_entities delta was invalid
} frame_t;
```

## ¿Cómo lo hace?
![](imagenes/quakeScheme.png)

```C
void CL_PredictMove (void) {
// .
// .
// .
// this is the last frame received from the server
	from = &cl.frames[cls.netchan.incoming_sequence & UPDATE_MASK];
// .
// .
// .

for (i=1 ; i<UPDATE_BACKUP-1 && cls.netchan.incoming_sequence+i <
			cls.netchan.outgoing_sequence; i++)
	{
		to = &cl.frames[(cls.netchan.incoming_sequence+i) & UPDATE_MASK];
		CL_PredictUsercmd (&from->playerstate[cl.playernum]
			, &to->playerstate[cl.playernum], &to->cmd, cl.spectator);
		if (to->senttime >= cl.time)
			break;
		from = to;
	}
}
//
```
<sub><sup>Creannos<sup><sub>

##### Ahora si
# Rollback Netcode
El concepto aparece a mediados de los 2000 y fue popularizado por la librería GGPO, desarrollada por Tony Cannon y vio la luz en 2006.

![](imagenes/tonycannon.png)

![](imagenes/ggpo.png) 

# La idea
- Ejecutar inmediatamente las entradas del jugador local
- Predecir las entradas de los jugadores remotos envés de esperarlos
- Una vez que nos llegan las entradas remotas
- Si la predicción es correcta -> el juego continua como si no pasó nada
- Si es incorrecta -> se revierte el estado del juego (rollback) y se continua desde el estado corregido

## GGPO
#### Good Game, Pace Out
<sub><sup>Traducido: Buen juego, nos vemos</sub></sup>
### ¿Qué hace?
Es un middleware que permite implementar rollback netcode en el multijugador de juegos, principalmente en juegos de pelea con multijugador P2P.

# ¿Y esto anda?

Por supuesto.

# Antes de avanzar, hay que aclarar un par de cosas:

- 30 FPS --> 1 Frame = 1/30s = 0,03s
- 60 FPS --> 1 Frame = 1/60s = 0,016s
- Tiempo de reacción promedio de una persona = 200ms = 0,2s 

- un frame es simulacion + renderizado:
    - los ticks son la unidad de tiempo en el que ocurren acciones de un juego, 
     la tasa a la cual la simulacion es ejecutada en un servidor se denomina tickrate, que son:
        * Inputs
        * Logica del juego
        * Fisicas, IA del juego, eventos
        
    - el renderizado es una visualizacion instantanea de la simulacion, una proyeccion del estado interno 

# Funcionamiento
![](imagenes/ggpo-func.png)

# ¿Cómo lo hace?
![](imagenes/ggpo-arch.png)

# ¿Cómo se hacen las predicciones?
Todos los inputs se guardan en una cola.

```C++
#define PREVIOUS_FRAME(offset)   (((offset) == 0) ? (INPUT_QUEUE_LENGTH - 1) : ((offset) - 1))

bool
InputQueue::GetInput(int requested_frame, GameInput *input){
    
    //En el medio hace otras cosas
    
/*
       * The requested frame isn't in the queue.  Bummer.  This means we need
       * to return a prediction frame.  Predict that the user will do the
       * same thing they did last time.
       */
      if (requested_frame == 0) {
         Log("basing new prediction frame from nothing, you're client wants frame 0.\n");
         _prediction.erase();
      } else if (_last_added_frame == GameInput::NullFrame) {
         Log("basing new prediction frame from nothing, since we have no frames yet.\n");
         _prediction.erase();
      } else {
         Log("basing new prediction frame from previously added frame (queue entry:%d, frame:%d).\n",
              PREVIOUS_FRAME(_head), _inputs[PREVIOUS_FRAME(_head)].frame);
         _prediction = _inputs[PREVIOUS_FRAME(_head)];
      }
      _prediction.frame++;
   }
```

# La comunidad dio su veredicto
| Veredicto:|
|----------|
| ![](imagenes/rant1.png) |
| ![](imagenes/rant2.png) |
| ![](imagenes/rant3.png) |
| ![](imagenes/rant4.png) |
| ![](imagenes/rant5.png) |
| ![](imagenes/rant6.png) |


# HERMOSO, BRILLANTE, INGENIOSO, pero…
¿Por qué a pesar del cariño de la comunidad el rollback netcode no es el estándar de los juegos de pelea multijugador?<br>
Parece ser que los desarrolladores japoneses no están muy abiertos a incorporar tecnologías hechas fuera de la isla.

# No todo es color de rosas
La predicción que hace el rollback es muy ingenua. Esto se hace presente cuando hay desconexiones largas ya que rollbacks se vuelven muy notorios porque los personajes dentro del juego se teletransportan por toda la pantalla.

Esto es una consecuencia directa de como funciona Rollback, ya que al hacer las simulaciones sin renderizar, de golpe tenes una renderizacion completamente diferente a lo que veias frames antes

# Un nuevo tipo de rollback
Esta alternativa implementa un esquema híbrido que usa rollback mezclado con, para sorpresa de nadie, inteligencia artificial.
![](imagenes/hybridposter.png)

# ¿Cómo funciona?
![](imagenes/poster1.png)

![](imagenes/poster2.png)

![](imagenes/poster3.png)

[Demostración gráfica](https://dl.acm.org/doi/abs/10.1145/3532719.3543199)

## Eso no es todo
Rollback Híbrido contempla diversos escenarios excepcionales, como cuando el jugador utiliza otra cuenta de otro usuario o el mejora/modifica su estilo de juego. 

## ¿Qué tan bien funciona?
Los investigadores probaron implementar este sistema en un juego de pelea open source conocido como Footsies. Obtuvieron que la tasa de precisión fue del 75%, y puede ser mejor a medida que se recolecta más información.

# ¿Preguntas?
Esperamos que no.
<br>
<br>

# Referencias:
- https://es.wikipedia.org/wiki/Netcode
- https://fabiensanglard.net/quakeSource/quakeSourcePrediction.php
- https://fabiensanglard.net/quakeSource/johnc-log.aug.htm
- https://github.com/id-Software/Quake/
- https://github.com/pond3r/ggpo
- https://dl.acm.org/doi/abs/10.1145/3532719.3543199